In [ ]:
import random
import tensorflow as tf
import numpy as np
import re
import pandas as pd
import utils
import editdistance
import csv
import multiprocessing

# Define scoring

In [ ]:
def ss(a: str, b: str):
    """Similarity score for two strings a and b"""
    if pd.isnull(a) or pd.isnull(b):
        return 0
    if a==b:
        return 1.0
    return editdistance.eval(a, b) / max(len(a), len(b))

Names which will be used in training dataframe:

In [ ]:
score_columns="r_name r_birthp r_civil r_pos r_job m_gender d_birthy".split(" ")

In [ ]:
def score(a, b):
    return (ss(a.Navn, b.Navn),
            ss(a.Fødested, b.Fødested),
            ss(a.Civilstand, b.Civilstand),
            ss(a.Position, b.Position),
            ss(a.Erhverv, b.Erhverv),
            1.0 if a.Køn == b.Køn else 0.0,
            abs(a.Fødeår - b.Fødeår))

# Get some training data

In [ ]:
matches = pd.read_csv(str(utils.datadir / "links" / "matches.csv"), sep="|")

In [ ]:
matches

In [ ]:
df = pd.read_pickle("dataframe.pickled")

In [ ]:
df.dropna(subset=("Fødeår",), inplace=True)

In [ ]:
df.set_index(["FT", "Kipnr", "Løbenr"], inplace=True)

In [ ]:
df.sort_index(inplace=True)

Getting raw data for all matches:

In [ ]:
def to_scored_frame(d):
    lots = []
    for t in d.itertuples():
        try:
            # TODO: optimize access
            s = df.loc[[t[1:4], t[4:]]]
            a = s.iloc[0]
            b = s.iloc[1]
        except KeyError:
            continue
        lots.append(score(a,b) + (True,))
    return pd.DataFrame(lots, columns=score_columns + ["label"])

In [ ]:
positive = utils.parallelize(matches[:10000], to_scored_frame)

Run through some arbitrary non-matches

In [ ]:
import random
import itertools

In [ ]:
years = [1845, 1850, 1860, 1880, 1885]

In [ ]:
ssize = len(positive)
def generate_nonmatch_scores(_):
    lots = []
    s = df.loc[random.choice(years)].sample(ssize)
    for i in range(ssize-1):
        a = s.iloc[i]
        b = s.iloc[i+1]
        lots.append(score(a,b) + (False,))
    return lots

In [ ]:
with multiprocessing.Pool() as p:
    res = p.map(generate_nonmatch_scores, [None for i in range(multiprocessing.cpu_count())])

In [ ]:
negative = pd.DataFrame(list(itertools.chain(*res)), columns=score_columns + ["label"])

In [ ]:
del(res)

In [ ]:
df_train = pd.concat([positive, negative])

# Build feature thing

In [ ]:
navn = tf.feature_column.numeric_column("r_name")
fødested = tf.feature_column.numeric_column("r_birthp")
civilstand = tf.feature_column.numeric_column("r_civil")
position = tf.feature_column.numeric_column("r_pos")
erhverv = tf.feature_column.numeric_column("r_job")
køn = tf.feature_column.numeric_column("m_gender")
fødeår = tf.feature_column.numeric_column("d_birthy")

# Build model

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=[
    navn, fødested, civilstand, position, erhverv, køn, fødeår
], model_dir="model")

In [ ]:
input_fn_train = tf.estimator.inputs.pandas_input_fn(
    x=df_train,
    y=df_train.label,
    shuffle=True)

In [ ]:
model.train(input_fn=input_fn_train, steps=100)

In [ ]:
model.evaluate(input_fn=input_fn_train)

# Or other model

In [ ]:
model2 = tf.estimator.DNNClassifier(feature_columns=[
    navn, fødested, civilstand, position, erhverv, køn, fødeår
], model_dir="model2", hidden_units=[7,5,3])

In [ ]:
model2.train(input_fn_train)

In [ ]:
model2.evaluate(input_fn_train)